## Import libraries

In [1]:
import sqlite3
import os
import csv
print("sqlite, os and csv imported !")

## Set working directory

In [2]:
if(os.getcwd().split('\\')[-1] == "NO_XXX_EVENT_EXPORT"): #check if te woring directory is already set
    print("working directory already set")
else: #the directory needs to be changed
    os.chdir(os.getcwd()+"/NO_XXX_EVENT_EXPORT") # change the directory to the directory with the csvfiles  
    print("working directory is set to: "+ os.getcwd())

## Set variables

In [3]:
bridge_con = sqlite3.connect("bridges.db")
bridge_cur = bridge_con.cursor()
meteo_con = sqlite3.connect("meteo.db")
meteo_cur = meteo_con.cursor()
clean_bridge_con = sqlite3.connect("clean_bridges.db")
clean_bridge_cur = clean_bridge_con.cursor()
csv_file = "NO_ZST_EVENT_EXPORT_2017-01-30_00-00-00_2017-01-30_23-59-59.CSV" #random csv file to read the headers from
brugnummers = ('W5','WA','WB','WBH','WD','WE','WG','WGR','WH','WHE','WHO','WHT','WHY','WL','WO','WOB','WS','WT','WW','ZBS','ZST')
print("variables meteo_con, meteo_cur, bridge_con, bridge_cur, clean_bridge_con, clean_bridge_cur, csv_file and brugnummers are set!")

## create a list with headers matching the columnnames in the DataBase

In [4]:
with open(csv_file) as file:
    reader = csv.reader(file,  delimiter=";")
    for _ in range(23):
        next(reader); 
    headers = next(reader)[:-1]
headers[headers.index('Cmnt.')] = 'Comment'
headers[headers.index('Tijd came')] = 'Tijd_came'
headers[headers.index('Tijd went')] = 'Tijd_went'
headers[headers.index('Tijd ack')] = 'Tijd_ack'
headers[headers.index('Gebr.ID')] = 'UUID'
print("all headers are collected and corrected to match the columnames in the database!")

all headers are collected and corrected to match the columnames in the database!


## create a new database for the meteo data

In [5]:
meteo_cur.execute("CREATE TABLE IF NOT EXISTS data (Tijd text, Waarde text, Omschrijving text, UUID text, DPE text);")
print("database created for meteo data!")

database created for meteo data!


## create a list with all meteo descriptions

In [6]:
for brugnummer in brugnummers:
    bridge_cur.execute("SELECT Omschrijving FROM "+brugnummer+" WHERE [Omschrijving] LIKE '%meteo%'")
    meteo_list = []
    for _ in set(bridge_cur.fetchall()):
        meteo_list += [str(_)[2:-3]]
print(meteo_list)

['Meteo Actuele Windrichting', 'Meteo Gemiddelde Windkracht', 'Meteo Hoogste Windkracht', 'Meteo Commando Reset Storing', 'Meteo Gemiddelde Windrichting', 'Meteo Actuele Luchtdruk', 'Meteo Actuele Waterhoogte', 'Meteo Actuele Windkracht', 'Meteo Actuele Onderdoorvaarthoogte', 'Meteo Actuele Temperatuur', 'Meteo Actuele Luchtvochtigheid', 'Meteo Windstoot Actief']


## create a list with all meteo descriptions and counts

In [7]:
# for list_item in meteo_list:
#     counter = 0
#     for brugnummer in brugnummers:
#         bridge_cur.execute("SELECT count(*) FROM "+brugnummer+" WHERE [Omschrijving] == '"+list_item+"'")
#         counter += int(str(bridge_cur.fetchall())[2:-3])
#     print(list_item +": "+str(counter))

## select all meteo data and put it in the new database
### takes a looooooooooooooooooong time

In [8]:
for list_item in meteo_list:
    for brugnummer in brugnummers:
        meteo_data_list = bridge_cur.execute("SELECT * FROM "+brugnummer+" WHERE [Omschrijving] == '"+list_item+"'")
        for row in meteo_data_list:
            to_db = (row[0],row[3],row[5],row[12],row[13])
            meteo_cur.executemany("INSERT INTO data (Tijd, Waarde, Omschrijving, UUID, DPE) VALUES (?, ?, ?, ?, ?);", (to_db,))
    print(list_item+ " naar database geschreven.")
meteo_con.commit()
print("alle data is naar meteo.db gechreven!")

Meteo Actuele Windrichting naar database geschreven.
Meteo Gemiddelde Windkracht naar database geschreven.
Meteo Hoogste Windkracht naar database geschreven.
Meteo Commando Reset Storing naar database geschreven.
Meteo Gemiddelde Windrichting naar database geschreven.
Meteo Actuele Luchtdruk naar database geschreven.
Meteo Actuele Waterhoogte naar database geschreven.
Meteo Actuele Windkracht naar database geschreven.
Meteo Actuele Onderdoorvaarthoogte naar database geschreven.
Meteo Actuele Temperatuur naar database geschreven.
Meteo Actuele Luchtvochtigheid naar database geschreven.
Meteo Windstoot Actief naar database geschreven.
alle data is naar meteo.db gechreven!


## test if the splited meteo data works

In [9]:
# for list_item in meteo_list:
#     meteo_cur.execute("SELECT count(*) FROM data WHERE [Omschrijving] == '"+list_item+"'")
#     print(list_item +": "+str(meteo_cur.fetchall())[2:-3])

## close all connections with the databases

In [10]:
bridge_con.close()
clean_bridge_con.close()
meteo_con.close()

In [11]:
# MeteoData is altijd een PROCES volgens de logs